In [156]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,precision_recall_curve, \
roc_auc_score,roc_curve,recall_score,classification_report, f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.svm import OneClassSVM
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedShuffleSplit
from lightgbm import LGBMClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import AdaBoostClassifier
import imblearn

In [81]:
df = pd.read_csv('../input/iba-ml1-mid-project/train.csv')
df['credit_line_utilization'] = df['credit_line_utilization'].str.replace(',', '.').astype(float)
columns = df.iloc[:, 1:-1].columns
columns

Index(['age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more'],
      dtype='object')

In [82]:
X, y = df[columns], df.iloc[:, -1]
Counter(y)

Counter({0: 67148, 1: 5013})

In [83]:
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X, y)

In [84]:
Counter(y_rus)

Counter({0: 5013, 1: 5013})

In [149]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe9 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier())
    ])
    pipe9.fit(X_train, y_train)
    y_predicted=pipe9.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

0.7599259648034608
0.7416263091347186


In [150]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe9.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

0.8870829361544617


In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe10 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe10.fit(X_train, y_train)
    y_predicted=pipe10.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe10.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

## PCA GridSearch PolynomialFeature

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
# from sklearn.feature_selection import SelectKBest

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    pipe12 = Pipeline(steps=[
            ('preprocessing', ColumnTransformer(transformers=[
                ('numeric', Pipeline(steps=[
                    ('impute', SimpleImputer(strategy='mean')),
                    ('scale', StandardScaler())
                ]), columns),
            ])),
            ('classifier', BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20))
        ])
    params = {
        'classifier__n_estimators': [80, 100, 120],
    }

    RF_gs = GridSearchCV(pipe12, param_grid=params, scoring='roc_auc', cv=3)
    RF_gs.fit(X_train, y_train)
    y_predicted=RF_gs.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
#     recall.append(recall_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=RF_gs.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe14 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', StandardScaler()),
                ('poly', PolynomialFeatures())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe14.fit(X_train, y_train)
    y_predicted=pipe14.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe14.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe15 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', StandardScaler()),
            ]), columns),
        ])),
        ('pca', PCA()),
        ('classifier', VotingClassifier(estimators=[
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe15.fit(X_train, y_train)
    y_predicted=pipe15.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe15.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
import umap

In [ ]:
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
# X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_rus)

In [ ]:
umap_reducer = umap.UMAP(n_components=2)
train_embedding = umap_reducer.fit_transform(X_imputed[columns])
plt.figure(figsize=(14, 6))
plt.scatter(train_embedding[:, 0], train_embedding[:, 1], s=4, c=y_rus)
plt.show()

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
iso = IsolationForest()
yhat = iso.fit_predict(X_imputed)
mask = yhat != -1
X_iso, y_iso = X_imputed[mask], y_rus[mask]

In [ ]:
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns

In [ ]:
iso = IsolationForest()
ocs = OneClassSVM()
y_iso = iso.fit_predict(X_imputed)
y_ocs = ocs.fit_predict(X_imputed)

m_iso = y_iso != -1
m_ocs = y_ocs != -1

In [ ]:
X_iso, y_iso = X_imputed[m_iso], y_rus[m_iso]
X_ocs, y_ocs = X_imputed[m_ocs], y_rus[m_ocs]

In [ ]:
X_iso.shape

In [ ]:
y_iso.shape

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_iso, y_iso):
    X_train, X_test = X_iso.iloc[train_idx], X_iso.iloc[test_idx]
    y_train, y_test = y_iso.iloc[train_idx], y_iso.iloc[test_idx]
    
    pipe20 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier())
    ])
    pipe20.fit(X_train, y_train)
    y_predicted=pipe20.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_iso, y_iso):
    X_train, X_test = X_iso.iloc[train_idx], X_iso.iloc[test_idx]
    y_train, y_test = y_iso.iloc[train_idx], y_iso.iloc[test_idx]
    
    pipe22 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=30))
    ])
    pipe22.fit(X_train, y_train)
    y_predicted=pipe22.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe22.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = pd.concat([X_rus, y_rus], axis=1).corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
?df.corr()

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    pipe30 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', XGBClassifier())
    ])
    pipe30.fit(X_train, y_train)
    y_predicted=pipe30.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

 ### ClassifierChain || pipe model series 40

In [94]:
from eli5 import show_weights
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import cross_val_score

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (LGBMClassifier()))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe41 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (XGBClassifier()))
    ])
    pipe41.fit(X_train, y_train)
    y_predicted=pipe41.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [108]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=20))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

0.7172233222797434
0.632026783087212


In [87]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe42 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (CatBoostClassifier()))
    ])
    pipe42.fit(X_train, y_train)
    y_predicted=pipe42.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

Learning rate set to 0.026355
0:	learn: 0.6826877	total: 64.4ms	remaining: 1m 4s
1:	learn: 0.6727313	total: 71.6ms	remaining: 35.7s
2:	learn: 0.6635943	total: 77.4ms	remaining: 25.7s
3:	learn: 0.6546750	total: 82.9ms	remaining: 20.6s
4:	learn: 0.6467690	total: 88.1ms	remaining: 17.5s
5:	learn: 0.6384842	total: 93ms	remaining: 15.4s
6:	learn: 0.6308898	total: 98.5ms	remaining: 14s
7:	learn: 0.6237840	total: 105ms	remaining: 13s
8:	learn: 0.6175803	total: 110ms	remaining: 12.1s
9:	learn: 0.6113849	total: 116ms	remaining: 11.5s
10:	learn: 0.6055017	total: 122ms	remaining: 11s
11:	learn: 0.5999069	total: 128ms	remaining: 10.5s
12:	learn: 0.5946490	total: 134ms	remaining: 10.2s
13:	learn: 0.5896139	total: 140ms	remaining: 9.86s
14:	learn: 0.5854330	total: 146ms	remaining: 9.6s
15:	learn: 0.5810447	total: 153ms	remaining: 9.39s
16:	learn: 0.5766091	total: 159ms	remaining: 9.17s
17:	learn: 0.5729622	total: 165ms	remaining: 8.97s
18:	learn: 0.5692802	total: 170ms	remaining: 8.78s
19:	learn: 0.

183:	learn: 0.4678001	total: 1.18s	remaining: 5.25s
184:	learn: 0.4677316	total: 1.19s	remaining: 5.24s
185:	learn: 0.4675631	total: 1.2s	remaining: 5.24s
186:	learn: 0.4673811	total: 1.2s	remaining: 5.23s
187:	learn: 0.4672434	total: 1.21s	remaining: 5.22s
188:	learn: 0.4671699	total: 1.21s	remaining: 5.21s
189:	learn: 0.4670640	total: 1.22s	remaining: 5.2s
190:	learn: 0.4669070	total: 1.23s	remaining: 5.21s
191:	learn: 0.4667314	total: 1.24s	remaining: 5.22s
192:	learn: 0.4666526	total: 1.25s	remaining: 5.21s
193:	learn: 0.4664997	total: 1.25s	remaining: 5.21s
194:	learn: 0.4664491	total: 1.26s	remaining: 5.2s
195:	learn: 0.4662884	total: 1.26s	remaining: 5.19s
196:	learn: 0.4661745	total: 1.27s	remaining: 5.18s
197:	learn: 0.4661019	total: 1.28s	remaining: 5.18s
198:	learn: 0.4659589	total: 1.28s	remaining: 5.17s
199:	learn: 0.4658139	total: 1.29s	remaining: 5.17s
200:	learn: 0.4656538	total: 1.3s	remaining: 5.16s
201:	learn: 0.4654911	total: 1.3s	remaining: 5.15s
202:	learn: 0.4654

369:	learn: 0.4472109	total: 2.37s	remaining: 4.04s
370:	learn: 0.4471818	total: 2.38s	remaining: 4.04s
371:	learn: 0.4470929	total: 2.39s	remaining: 4.03s
372:	learn: 0.4469721	total: 2.39s	remaining: 4.02s
373:	learn: 0.4468140	total: 2.4s	remaining: 4.01s
374:	learn: 0.4466996	total: 2.4s	remaining: 4s
375:	learn: 0.4465360	total: 2.41s	remaining: 4s
376:	learn: 0.4463970	total: 2.42s	remaining: 3.99s
377:	learn: 0.4463799	total: 2.42s	remaining: 3.98s
378:	learn: 0.4462705	total: 2.43s	remaining: 3.98s
379:	learn: 0.4461291	total: 2.43s	remaining: 3.97s
380:	learn: 0.4459260	total: 2.44s	remaining: 3.96s
381:	learn: 0.4457658	total: 2.44s	remaining: 3.95s
382:	learn: 0.4456711	total: 2.45s	remaining: 3.95s
383:	learn: 0.4455084	total: 2.46s	remaining: 3.94s
384:	learn: 0.4453799	total: 2.46s	remaining: 3.93s
385:	learn: 0.4452936	total: 2.47s	remaining: 3.92s
386:	learn: 0.4451723	total: 2.47s	remaining: 3.92s
387:	learn: 0.4450670	total: 2.48s	remaining: 3.91s
388:	learn: 0.444976

538:	learn: 0.4266589	total: 3.36s	remaining: 2.88s
539:	learn: 0.4265657	total: 3.37s	remaining: 2.87s
540:	learn: 0.4264506	total: 3.38s	remaining: 2.86s
541:	learn: 0.4262663	total: 3.38s	remaining: 2.86s
542:	learn: 0.4261447	total: 3.39s	remaining: 2.85s
543:	learn: 0.4260745	total: 3.39s	remaining: 2.84s
544:	learn: 0.4259545	total: 3.4s	remaining: 2.84s
545:	learn: 0.4258271	total: 3.4s	remaining: 2.83s
546:	learn: 0.4256724	total: 3.41s	remaining: 2.83s
547:	learn: 0.4255882	total: 3.42s	remaining: 2.82s
548:	learn: 0.4254531	total: 3.42s	remaining: 2.81s
549:	learn: 0.4253069	total: 3.43s	remaining: 2.8s
550:	learn: 0.4251848	total: 3.43s	remaining: 2.8s
551:	learn: 0.4250662	total: 3.44s	remaining: 2.79s
552:	learn: 0.4249078	total: 3.44s	remaining: 2.78s
553:	learn: 0.4248524	total: 3.45s	remaining: 2.78s
554:	learn: 0.4246613	total: 3.46s	remaining: 2.77s
555:	learn: 0.4245986	total: 3.46s	remaining: 2.76s
556:	learn: 0.4245301	total: 3.47s	remaining: 2.76s
557:	learn: 0.42

712:	learn: 0.4074727	total: 4.35s	remaining: 1.75s
713:	learn: 0.4073820	total: 4.36s	remaining: 1.75s
714:	learn: 0.4072855	total: 4.37s	remaining: 1.74s
715:	learn: 0.4071896	total: 4.37s	remaining: 1.73s
716:	learn: 0.4070680	total: 4.38s	remaining: 1.73s
717:	learn: 0.4069703	total: 4.38s	remaining: 1.72s
718:	learn: 0.4068778	total: 4.39s	remaining: 1.72s
719:	learn: 0.4067908	total: 4.39s	remaining: 1.71s
720:	learn: 0.4067390	total: 4.4s	remaining: 1.7s
721:	learn: 0.4066347	total: 4.41s	remaining: 1.7s
722:	learn: 0.4065215	total: 4.41s	remaining: 1.69s
723:	learn: 0.4064194	total: 4.42s	remaining: 1.68s
724:	learn: 0.4063996	total: 4.42s	remaining: 1.68s
725:	learn: 0.4062788	total: 4.43s	remaining: 1.67s
726:	learn: 0.4061675	total: 4.43s	remaining: 1.67s
727:	learn: 0.4060672	total: 4.44s	remaining: 1.66s
728:	learn: 0.4060058	total: 4.45s	remaining: 1.65s
729:	learn: 0.4059568	total: 4.45s	remaining: 1.65s
730:	learn: 0.4058427	total: 4.46s	remaining: 1.64s
731:	learn: 0.4

...

238:	learn: 0.4598322	total: 1.45s	remaining: 4.61s
239:	learn: 0.4598142	total: 1.45s	remaining: 4.6s
240:	learn: 0.4597916	total: 1.46s	remaining: 4.6s
241:	learn: 0.4596307	total: 1.47s	remaining: 4.59s
242:	learn: 0.4595327	total: 1.47s	remaining: 4.58s
243:	learn: 0.4594531	total: 1.48s	remaining: 4.57s
244:	learn: 0.4593680	total: 1.48s	remaining: 4.57s
245:	learn: 0.4593047	total: 1.49s	remaining: 4.56s
246:	learn: 0.4591522	total: 1.49s	remaining: 4.55s
247:	learn: 0.4590164	total: 1.5s	remaining: 4.54s
248:	learn: 0.4589473	total: 1.5s	remaining: 4.54s
249:	learn: 0.4588687	total: 1.51s	remaining: 4.53s
250:	learn: 0.4587348	total: 1.51s	remaining: 4.52s
251:	learn: 0.4586233	total: 1.52s	remaining: 4.51s
252:	learn: 0.4585273	total: 1.52s	remaining: 4.5s
253:	learn: 0.4584011	total: 1.53s	remaining: 4.5s
254:	learn: 0.4582936	total: 1.54s	remaining: 4.49s
255:	learn: 0.4582452	total: 1.54s	remaining: 4.48s
256:	learn: 0.4580729	total: 1.55s	remaining: 4.47s
257:	learn: 0.4579

409:	learn: 0.4408521	total: 2.44s	remaining: 3.5s
410:	learn: 0.4407287	total: 2.44s	remaining: 3.5s
411:	learn: 0.4405366	total: 2.45s	remaining: 3.49s
412:	learn: 0.4404416	total: 2.45s	remaining: 3.49s
413:	learn: 0.4402856	total: 2.46s	remaining: 3.48s
414:	learn: 0.4401760	total: 2.47s	remaining: 3.48s
415:	learn: 0.4401099	total: 2.47s	remaining: 3.47s
416:	learn: 0.4399246	total: 2.48s	remaining: 3.46s
417:	learn: 0.4398038	total: 2.48s	remaining: 3.46s
418:	learn: 0.4396950	total: 2.49s	remaining: 3.45s
419:	learn: 0.4395323	total: 2.5s	remaining: 3.45s
420:	learn: 0.4394002	total: 2.5s	remaining: 3.44s
421:	learn: 0.4393284	total: 2.51s	remaining: 3.44s
422:	learn: 0.4391623	total: 2.52s	remaining: 3.43s
423:	learn: 0.4390521	total: 2.52s	remaining: 3.43s
424:	learn: 0.4389022	total: 2.53s	remaining: 3.42s
425:	learn: 0.4388011	total: 2.53s	remaining: 3.41s
426:	learn: 0.4386832	total: 2.54s	remaining: 3.41s
427:	learn: 0.4385173	total: 2.54s	remaining: 3.4s
428:	learn: 0.438

576:	learn: 0.4203058	total: 3.42s	remaining: 2.51s
577:	learn: 0.4202353	total: 3.43s	remaining: 2.5s
578:	learn: 0.4200535	total: 3.44s	remaining: 2.5s
579:	learn: 0.4199593	total: 3.44s	remaining: 2.49s
580:	learn: 0.4198561	total: 3.45s	remaining: 2.49s
581:	learn: 0.4197587	total: 3.45s	remaining: 2.48s
582:	learn: 0.4196407	total: 3.46s	remaining: 2.47s
583:	learn: 0.4194883	total: 3.46s	remaining: 2.47s
584:	learn: 0.4193640	total: 3.47s	remaining: 2.46s
585:	learn: 0.4192393	total: 3.47s	remaining: 2.45s
586:	learn: 0.4191328	total: 3.48s	remaining: 2.45s
587:	learn: 0.4190489	total: 3.49s	remaining: 2.44s
588:	learn: 0.4189655	total: 3.49s	remaining: 2.44s
589:	learn: 0.4188234	total: 3.5s	remaining: 2.43s
590:	learn: 0.4186751	total: 3.5s	remaining: 2.42s
591:	learn: 0.4185649	total: 3.51s	remaining: 2.42s
592:	learn: 0.4184115	total: 3.51s	remaining: 2.41s
593:	learn: 0.4183511	total: 3.52s	remaining: 2.4s
594:	learn: 0.4182831	total: 3.52s	remaining: 2.4s
595:	learn: 0.4181

746:	learn: 0.4023413	total: 4.41s	remaining: 1.49s
747:	learn: 0.4022546	total: 4.42s	remaining: 1.49s
748:	learn: 0.4021618	total: 4.42s	remaining: 1.48s
749:	learn: 0.4021138	total: 4.43s	remaining: 1.48s
750:	learn: 0.4019822	total: 4.43s	remaining: 1.47s
751:	learn: 0.4018496	total: 4.44s	remaining: 1.46s
752:	learn: 0.4017709	total: 4.44s	remaining: 1.46s
753:	learn: 0.4016989	total: 4.45s	remaining: 1.45s
754:	learn: 0.4016574	total: 4.45s	remaining: 1.45s
755:	learn: 0.4015289	total: 4.46s	remaining: 1.44s
756:	learn: 0.4014421	total: 4.46s	remaining: 1.43s
757:	learn: 0.4013853	total: 4.47s	remaining: 1.43s
758:	learn: 0.4012203	total: 4.48s	remaining: 1.42s
759:	learn: 0.4010576	total: 4.48s	remaining: 1.42s
760:	learn: 0.4009447	total: 4.49s	remaining: 1.41s
761:	learn: 0.4008537	total: 4.49s	remaining: 1.4s
762:	learn: 0.4007619	total: 4.5s	remaining: 1.4s
763:	learn: 0.4005572	total: 4.5s	remaining: 1.39s
764:	learn: 0.4004526	total: 4.51s	remaining: 1.39s
765:	learn: 0.40

915:	learn: 0.3866501	total: 5.38s	remaining: 494ms
916:	learn: 0.3865467	total: 5.39s	remaining: 488ms
917:	learn: 0.3864743	total: 5.4s	remaining: 482ms
918:	learn: 0.3863893	total: 5.41s	remaining: 476ms
919:	learn: 0.3863260	total: 5.41s	remaining: 470ms
920:	learn: 0.3862070	total: 5.42s	remaining: 465ms
921:	learn: 0.3861064	total: 5.42s	remaining: 459ms
922:	learn: 0.3860479	total: 5.43s	remaining: 453ms
923:	learn: 0.3859350	total: 5.43s	remaining: 447ms
924:	learn: 0.3858319	total: 5.44s	remaining: 441ms
925:	learn: 0.3857337	total: 5.44s	remaining: 435ms
926:	learn: 0.3856331	total: 5.45s	remaining: 429ms
927:	learn: 0.3855117	total: 5.45s	remaining: 423ms
928:	learn: 0.3854229	total: 5.46s	remaining: 417ms
929:	learn: 0.3853058	total: 5.46s	remaining: 411ms
930:	learn: 0.3852011	total: 5.47s	remaining: 405ms
931:	learn: 0.3851347	total: 5.48s	remaining: 400ms
932:	learn: 0.3850135	total: 5.48s	remaining: 394ms
933:	learn: 0.3849079	total: 5.49s	remaining: 388ms
934:	learn: 0

In [130]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe43 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier(n_estimators=200, learning_rate=0.5,max_depth=1, random_state=0))
    ])
    pipe43.fit(X_train, y_train)
    y_predicted=pipe43.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

0.761036890362701
0.7263353770546557


In [107]:
cv = KFold(n_splits=8, random_state=1, shuffle=True)
# create model
# model = DecisionTreeClassifier()
# evaluate model
scores = cross_val_score(pipe9, X_rus, y_rus, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy:',np.mean(scores))

Accuracy: 0.757929613266279


In [131]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe43.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

0.7685967002827342


### Dimensionality reduction || pipe version 50

In [110]:
from sklearn.decomposition import PCA

In [112]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(SimpleImputer(strategy='mean').fit_transform(X_rus))

In [120]:
cv = StratifiedKFold(n_splits=8, random_state=1, shuffle=True)
# create model
model = DecisionTreeClassifier()
# evaluate model
scores = cross_val_score(pipe9, X_rus, y_rus, scoring='roc_auc', cv=cv, n_jobs=-1)
# report performance
print('Accuracy:',np.mean(scores))

Accuracy: 0.8336129581237133


In [126]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_pca, y_rus):
    X_train, X_test = X_pca[train_idx], X_pca[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (LGBMClassifier()))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [122]:
test = pd.read_csv('../input/iba-ml1-mid-project/test.csv')
test['credit_line_utilization'] = test['credit_line_utilization'].str.replace(',', '.').astype(float)
test_prediction = pipe9z.predict(test.iloc[:, 1:])
submission = pd.DataFrame({'Predicted':test_prediction})
submission = submission.set_index(pd.Index(np.arange(1, 48109))).reset_index()

In [123]:
submission.to_csv('submission9z.csv')

In [ ]:
perm = PermutationImportance(pipe40, scoring = 'roc_auc'
,random_state=101).fit(X_test, y_test)
show_weights(perm, feature_names = list(X_test.columns))

### Outlier removed || Pipe 60 

In [159]:
!pip install smote

In [172]:
from imblearn.over_sampling import SMOTE

In [168]:
from sklearn.ensemble import IsolationForest
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
iso = IsolationForest()
yhat = iso.fit_predict(X_imputed)
mask = yhat != -1
X_iso, y_iso = X_imputed[mask], y[mask]

In [169]:
print(X_imputed.shape)
print(X_iso.shape)

(72161, 10)
(67795, 10)


In [175]:
Counter(y_iso)

Counter({0: 64243, 1: 3552})

In [176]:
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = imblearn.pipeline.Pipeline(steps=steps)
X_smt, y_smt = pipeline.fit_resample(X_iso, y_iso)
Counter(y_smt)

Counter({0: 12848, 1: 6424})

In [177]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipe60 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier())
    ])
    pipe60.fit(X_train, y_train)
    y_predicted=pipe60.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

0.7920393220010771
0.6589424572317263


In [179]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_iso, y_iso) 
    y_predicted=pipe60.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

0.9409957549296077


In [164]:
Counter(y_iso)

Counter({0: 4899, 1: 4600})

In [ ]:
ros = RandomOverSampler(sampling_strategy=0.5)
rus = RandomUnderSampler(sampling_strategy=0.8)

X_ros, y_ros = ros.fit_resample(X_iso, y_iso)
X_comb, y_comb = rus.fit_resample(X_ros, y_ros)
Counter(y_ros)

In [180]:
test = pd.read_csv('../input/iba-ml1-mid-project/test.csv')
test['credit_line_utilization'] = test['credit_line_utilization'].str.replace(',', '.').astype(float)
test_prediction = pipe60.predict(test.iloc[:, 1:])
submission = pd.DataFrame({'Predicted':test_prediction})
submission = submission.set_index(pd.Index(np.arange(1, 48109))).reset_index()

NameError: name 'pipe60x' is not defined

In [ ]:
submission.to_csv('submission60x.csv')